In [6]:
import pandas as pd
import numpy as np

teams = pd.read_csv("data/teams.csv")
games = pd.read_csv("games/games.csv")

teams = teams[['Rk', 'Team', 'Conf', 'AdjEM', 'AdjO',
       'AdjO_Rk', 'AdjD', 'AdjD_Rk', 'AdjT', 'AdjT_Rk', 'date']]
games = games[['Date', 'Home Team', 'Away Team',
       'Home 1H Points', 'Away 1H Points', 'Home Full Points',
       'Away Full Points']]

teams['date'] = pd.to_datetime(teams['date']).dt.date
games['Date'] = pd.to_datetime(games['Date']).dt.date

In [7]:
teams['Team'] = teams['Team'].str.replace(r'\d+', '', regex=True).str.strip()
teams = teams[teams['Team'] != "Team"]

In [8]:
games['Home Team'] = games['Home Team'].str.replace(r'\d+', '', regex=True).str.strip()
games['Away Team'] = games['Away Team'].str.replace(r'\d+', '', regex=True).str.strip()

In [9]:
teams['date'] = pd.to_datetime(teams['date'])

# Sort by date
teams = teams.sort_values(by=['Team', 'date']).reset_index(drop=True)

# Define columns for rolling calculations
numeric_columns = ['Rk', 'AdjEM', 'AdjO', 'AdjO_Rk', 'AdjD', 'AdjD_Rk', 'AdjT', 'AdjT_Rk']

# Define rolling windows
rolling_windows = [1, 3, 5, 10, 20]

# Create rolling features
for window in rolling_windows:
    for col in numeric_columns:
        # Create rolling mean feature
        teams[f'{col}_rolling_{window}'] = (
            teams.groupby('Team')[col]  # Group by 'Team'
            .transform(lambda x: x.rolling(window, min_periods=1).mean())
        )

In [11]:
teams

,Rk,Team,Conf,AdjEM,AdjO,AdjO_Rk,AdjD,AdjD_Rk,AdjT,AdjT_Rk,...,AdjT_rolling_10,AdjT_Rk_rolling_10,Rk_rolling_20,AdjEM_rolling_20,AdjO_rolling_20,AdjO_Rk_rolling_20,AdjD_rolling_20,AdjD_Rk_rolling_20,AdjT_rolling_20,AdjT_Rk_rolling_20
0,322,Abilene Christian,Slnd,-13.53,93.8,304,107.4,329,70.7,181,...,70.700000,181.000000,322.00,-13.5300,93.800000,304.000000,107.400000,329.00,70.700000,181.000000
1,323,Abilene Christian,Slnd,-13.53,94.6,303,108.2,328,72.3,189,...,71.500000,185.000000,322.50,-13.5300,94.200000,303.500000,107.800000,328.50,71.500000,185.000000
2,324,Abilene Christian,Slnd,-13.53,94.6,303,108.2,327,72.3,192,...,71.766667,187.333333,323.00,-13.5300,94.333333,303.333333,107.933333,328.00,71.766667,187.333333
3,324,Abilene Christian,Slnd,-13.53,95.0,303,108.5,327,72.4,189,...,71.925000,187.750000,323.25,-13.5300,94.500000,303.250000,108.075000,327.75,71.925000,187.750000
4,324,Abilene Christian,Slnd,-13.53,95.8,304,109.4,328,72.4,189,...,72.020000,188.000000,323.40,-13.5300,94.760000,303.400000,108.340000,327.80,72.020000,188.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397687,134,Youngstown St.,Horz,+3.02,108.3,138,105.3,153,69.0,94,...,68.960000,94.900000,134.15,3.0560,108.320000,140.350000,105.260000,152.40,68.935000,94.750000
397688,134,Youngstown St.,Horz,+3.03,108.3,138,105.3,153,69.0,94,...,68.970000,94.700000,134.05,3.0600,108.305000,140.100000,105.245000,152.40,68.935000,94.800000
397689,134,Youngstown St.,Horz,+3.02,108.3,138,105.3,153,69.0,94,...,68.980000,94.600000,133.90,3.0630,108.295000,139.850000,105.230000,152.40,68.930000,94.850000
397690,134,Youngstown St.,Horz,+3.02,108.3,138,105.3,153,69.0,94,...,68.990000,94.500000,133.75,3.0650,108.285000,139.600000,105.220000,152.35,68.930000,94.800000


In [15]:
from datetime import timedelta
# Ensure date columns are in datetime format
games['Date'] = pd.to_datetime(games['Date'])
teams['date'] = pd.to_datetime(teams['date'])
# Function to merge games with rankings using date and team names with date tolerance
def merge_with_tolerance(games_df, rankings_df, days_tolerance=2):
    # Merge exact date matches
    merged_df = games_df.merge(rankings_df, how='left', left_on=['Date', 'Home Team'], right_on=['date', 'Team'])
    merged_df = merged_df.merge(rankings_df, how='left', left_on=['Date', 'Away Team'], right_on=['date', 'Team'], suffixes=('_home', '_away'))

    # Try matching dates up to `days_tolerance` days before if no exact match is found
    for days_back in range(1, days_tolerance + 1):
        unmatched_home = merged_df[merged_df['Rk_home'].isna()]
        unmatched_away = merged_df[merged_df['Rk_away'].isna()]

        # Home team fallback match
        fallback_home = games_df[games_df['Date'].isin(unmatched_home['Date'] - timedelta(days=days_back))]
        fallback_merged_home = fallback_home.merge(rankings_df, how='left', left_on=['Date', 'Home Team'], right_on=['date', 'Team'])
        
        # Away team fallback match
        fallback_away = games_df[games_df['Date'].isin(unmatched_away['Date'] - timedelta(days=days_back))]
        fallback_merged_away = fallback_away.merge(rankings_df, how='left', left_on=['Date', 'Away Team'], right_on=['date', 'Team'])

        # Update original DataFrame with fallback data
        merged_df.update(fallback_merged_home)
        merged_df.update(fallback_merged_away)
    
    return merged_df

# Apply the function
merged_df = merge_with_tolerance(games, teams)

# Display the merged DataFrame
print(merged_df)

            Date         Home Team     Away Team  Home 1H Points  \
0     2012-11-09               TCU      Cal Poly            20.0   
1     2012-11-09           Belmont      Lipscomb            26.0   
2     2012-11-09        Kansas St.  North Dakota            36.0   
3     2012-11-09           Houston   Florida A&M            34.0   
4     2012-11-09             IUPUI   Utah Valley            43.0   
...          ...               ...           ...             ...   
39796 2024-03-11  South Dakota St.    St. Thomas            28.0   
39797 2024-03-13          Maryland       Rutgers            36.0   
39798 2024-03-13          Penn St.      Michigan            33.0   
39799 2024-03-14      Michigan St.     Minnesota            32.0   
39800 2024-03-14          Ohio St.          Iowa            38.0   

       Away 1H Points  Home Full Points  Away Full Points Rk_home  \
0                21.0              53.0              46.0     NaN   
1                42.0              89.0      

/var/folders/0b/h0zc244107jg60tttkl4r0kw0000gn/T/ipykernel_2712/2282745401.py:25: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  merged_df.update(fallback_merged_home)
/var/folders/0b/h0zc244107jg60tttkl4r0kw0000gn/T/ipykernel_2712/2282745401.py:26: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  merged_df.update(fallback_merged_away)
/var/folders/0b/h0zc244107jg60tttkl4r0kw0000gn/T/ipykernel_2712/2282745401.py:25: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. 

In [20]:
model_df = merged_df.dropna()

In [21]:
model_df

,Date,Home Team,Away Team,Home 1H Points,Away 1H Points,Home Full Points,Away Full Points,Rk_home,Team_home,Conf_home,...,AdjT_rolling_10_away,AdjT_Rk_rolling_10_away,Rk_rolling_20_away,AdjEM_rolling_20_away,AdjO_rolling_20_away,AdjO_Rk_rolling_20_away,AdjD_rolling_20_away,AdjD_Rk_rolling_20_away,AdjT_rolling_20_away,AdjT_Rk_rolling_20_away
6503,2012-12-08,Chicago St.,Alabama St.,34.0,35.0,67.0,54.0,77,College of Charleston,CAA,...,79.10,1.0,330.00,-15.1200,95.800,262.00,110.900,351.00,79.100,1.00
6504,2012-12-08,Arkansas St.,St. Bonaventure,42.0,28.0,73.0,70.0,151,Massachusetts,A10,...,73.60,31.0,279.00,-8.7800,97.100,243.00,105.900,306.00,73.600,31.00
6505,2012-12-08,Duke,Temple,46.0,32.0,90.0,67.0,51,Pittsburgh,ACC,...,71.60,114.0,88.00,6.5600,103.600,89.00,97.000,98.00,71.600,114.00
6506,2012-12-08,West Virginia,Virginia Tech,30.0,24.0,68.0,67.0,1,Duke,ACC,...,71.10,152.0,242.00,-5.9800,99.700,171.00,105.600,301.00,71.100,152.00
6507,2012-12-08,Virginia,Mississippi Valley St.,36.0,22.0,67.0,39.0,62,South Carolina,SEC,...,73.60,30.0,169.00,-0.4700,99.300,185.00,99.800,148.00,73.600,30.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39796,2024-03-11,South Dakota St.,St. Thomas,28.0,25.0,59.0,49.0,141,South Dakota St.,Sum,...,63.55,350.7,148.95,1.4665,108.510,142.10,107.040,182.40,63.450,350.10
39797,2024-03-13,Maryland,Rutgers,36.0,22.0,65.0,51.0,59,Maryland,B10,...,67.94,166.5,90.75,8.9600,100.915,280.85,91.955,3.75,68.030,162.45
39798,2024-03-13,Penn St.,Michigan,33.0,22.0,66.0,57.0,77,Penn St.,B10,...,67.92,168.0,118.60,5.1130,112.165,84.55,107.065,180.80,67.760,180.35
39799,2024-03-14,Michigan St.,Minnesota,32.0,34.0,77.0,67.0,19,Michigan St.,B10,...,66.90,234.5,68.35,11.6625,114.330,56.10,102.655,99.85,67.000,230.50


In [22]:
model_df.to_csv("model_df.csv")